In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import sys
import pickle

In [2]:
current_path = '/Users/bit/Desktop/hyunmi/'

In [6]:
# Load movies data
movies = pd.read_csv('./data/movies.csv')
genome_scores = pd.read_csv('./data/genome-scores.csv')
tags = pd.read_csv('./data/tags.csv')
genome_tags = pd.read_csv('./data/genome-tags.csv')
# Use ratings data to downsample tags data to only movies with ratings 
ratings = pd.read_csv('./data/ratings.csv')
#ratings = ratings.drop_duplicates('movieId')

In [7]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

In [9]:
ratings_f = ratings.groupby('userId').filter(lambda x: len(x) >= 55)
# instantiate a reader and read in our rating data
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_f[['userId','movieId','rating']], reader)

# train SVD on 75% of known rates
trainset, testset = train_test_split(data, test_size=.25)
algorithm = SVD()
algorithm.fit(trainset)
predictions = algorithm.test(testset)

# check the accuracy using Root Mean Square Error
accuracy.rmse(predictions)

RMSE: 0.7688


0.7687718650374417

In [ ]:
movie_list_rating = ratings_f.movieId.unique().tolist()
movies = movies[movies.movieId.isin(movie_list_rating)]
Mapping_file = dict(zip(movies.title.))

In [10]:
def pred_user_rating(ui):
    if ui in ratings_f.userId.unique():
        ui_list = ratings_f[ratings_f.userId == ui].movieId.tolist()
        d = {k: v for k,v in Mapping_file.items() if not v in ui_list}        
        predictedL = []
        for i, j in d.items():     
            predicted = algorithm.predict(ui, j)
            predictedL.append((i, predicted[3])) 
        pdf = pd.DataFrame(predictedL, columns = ['movies', 'ratings'])
        pdf.sort_values('ratings', ascending=False, inplace=True)  
        pdf.set_index('movies', inplace=True)    
        return pdf.head(10)        
    else:
        print("User Id does not exist in the list!")
        return None

In [12]:
user_id = 70
pred_user_rating(user_id)

NameError: name 'Mapping_file' is not defined